In [2]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

In [6]:
today = date.today()
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-08-22
yesterday: 2025-08-21


In [8]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [10]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Tables in the process

In [13]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [15]:
def show_buy(const, name):
    sql = """
    SELECT * 
    FROM buy 
    WHERE name = '{}'
    """.format(name)
    
    df_buy = pd.read_sql(sql, const)
    df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
    df_buy.rename(columns={'volbuy':'shares'},inplace=True)
    df_buy['shares'] = df_buy['shares'].astype('int64')
    return df_buy.style.format(format_dict)

In [17]:
def show_dividend(const, name):
    sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
    """.format(name)
    
    df_dividend = pd.read_sql(sql, const)
    df_dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
    df_dividend.columns = df_dividend.columns.str.lower()
    df_dividend['shares'] = df_dividend['shares'].astype('int64')
    df_dividend['xdate'] = pd.to_datetime(df_dividend['xdate'])
    df_dividend['paiddate'] = pd.to_datetime(df_dividend['paiddate'])
    return df_dividend.style.format(format_dict)

In [19]:
def show_stocks(conlite, name):
    sql = """
    SELECT * 
    FROM stocks 
    WHERE name = '{}'
    """.format(name)
    
    df_stocks = pd.read_sql(sql, conlite)
    return df_stocks.style.format(format_dict)

In [35]:
def update_buy(const, name, new_qty, new_price):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty_value,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [39]:
def update_buy(const, name, new_qty, new_price):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [50]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [58]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty)
    new_unit_cost_value = float(new_unit_cost)   
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    result = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {result.rowcount}"

### End of Definition

In [72]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
df_sells_latest = pd.read_sql(sql, conpf)
df_sells_latest.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,CPF,21,2025-08-22,"5,000",24.10,1109,1342,2025-08-22,24.30,251.51,17.61,"121,230.88",1,463.98,0.38%,140.23%,-109,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [92]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = df_sells_latest['name']
name = names.to_string(index=False)
name

'CPF'

In [108]:
sr_qty = df_sells_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = df_sells_latest['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = df_sells_latest['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = df_sells_latest['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

df_buys_latest = pd.read_sql(sql, conpf)
df_buys_latest.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1342



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,CPF,1342,21,2025-08-22,"5,000",24.10,249.44,17.46,Sold,"120,766.90",DTD,None


In [110]:
transaction, name, qty, sell_price, buy_price

('B', 'CPF', -5000, 24.3, 24.1)

### In case of buy, retreive buy id from latest buy record (id)

In [23]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
df_buys_latest = pd.read_sql(sql, conpf)
df_buys_latest.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,CPF,1342,21,2025-08-22,"5,000",24.10,249.44,17.46,Sold,"120,766.90",DTD,None


In [104]:
transaction = 'B'

names = df_buys_latest['name']
name = names.to_string(index=False)

sr_qty = df_buys_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = df_buys_latest['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'CPF', 5000, 24.1)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [125]:
old_qty = df_buys_latest['qty']
old_qty

0    5000
Name: qty, dtype: int64

In [127]:
new_qty = old_qty + qty
print(new_qty)

0    0
Name: qty, dtype: int64


In [129]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,CPF,2025-08-21,"5,000",24.10,1,4,A1


In [137]:
buy_price = 24.1
buy_price

24.1

In [139]:
print(transaction, name, new_qty, buy_price)

B CPF 0    0
Name: qty, dtype: int64 24.1


In [143]:
# Example usage
update_buy(const, name, new_qty, buy_price)

'Records updated = 1'

In [145]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,CPF,2025-08-21,0,24.10,1,4,A1


In [37]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 1


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [147]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,CPF,0.5500,0.0000,1.0000,0.0000,1.5500,"5,000",2025-08-29,2025-09-12,,1


In [149]:
print(new_qty)

0    0
Name: qty, dtype: int64


In [151]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [45]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


In [153]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,CPF,0.5500,0.0000,1.0000,0.0000,1.5500,0,2025-08-29,2025-09-12,,1


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [155]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,152,CPF,24.40,23.90,B,24.10,24.30,"4,000.00",0.00,24.10,"5,000",-4,4,"5,000",0.000000,0.000000,,SET50


In [159]:
# Extract scalar values from Pandas Series (if applicable)
new_qty_value = new_qty.iloc[0] if hasattr(new_qty, 'iloc') else new_qty
#cost_value = cost.iloc[0] if hasattr(cost, 'iloc') else cost
# Print the values
print(name, transaction, new_qty_value, buy_price)

CPF B 0 24.1


In [102]:
#print("Raw cost value before extraction:", cost)

In [104]:
def update_stock(conlite, name, new_qty, new_price):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty,
        cost = :new_price
        WHERE name = :name
    """)    
    # Execute the query with parameters
    result = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value, # Use the scalar value
        'new_price': new_price,
        'name': name
    })
    return f"Records updated = {result.rowcount}"

In [ ]:
new_buy_target = 24.1
new_sell_target = 24.3
new_buy_qty = 5000

In [167]:
print(new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

0    0
Name: qty, dtype: int64 24.1 24.1 24.3 5000


In [169]:
# Example usage
update_stock(conlite, name, new_qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

C:\Users\PC1\AppData\Local\Temp\ipykernel_218744\2826900856.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  new_qty_value = int(new_qty)


'Records updated = 1'

In [171]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,152,CPF,24.40,23.90,B,24.10,24.30,"4,000.00",0.00,24.10,"5,000",-4,4,0,0.000000,0.000000,,SET50


In [103]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM stocks
    WHERE name = :name AND available_qty = 0;
""")

# Execute the query with the correct parameter dictionary
result = conlite.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [27]:
sql = """
SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

df_buy = pd.read_sql(sql, const)
df_buy['date'] = pd.to_datetime(df_buy['date'])
df_buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
df_buy.rename(columns={'volbuy':'shares'}, inplace = True)
df_buy['shares'] = df_buy['shares'].astype('int64')
df_buy


SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = 'CPF'



,name,date,shares,price,active,dividend,period,grade,cost_amt
0,CPF,2025-08-21,0,24.1,1,1.55,4,A1,0.0


In [29]:
print(transaction, name, qty, buy_price)

B CPF 5000 24.1


In [32]:
#new_unit_cost = df_buy['cost_amt'].iloc[0] + (qty * buy_price)
new_unit_cost = 24.10
#new_qty = df_buy['shares'].iloc[0] + qty
new_qty = 4_000
print(new_unit_cost)
print(new_qty)
#new_unit_cost = new_unit_cost / new_qty
#print(new_unit_cost)

24.1
4000


### In case there is the same stock name in port

In [41]:
update_buy(const, name, qty, buy_price)

'Records updated = 1'

In [43]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,CPF,2025-08-21,"5,000",24.10,1,4,A1


### In case of totally new stock

In [36]:
rcd = {
    'name': 'CPF',
    'date': '2025-08-21',
    'volbuy': 4000,
    'price': 24.1,
    'volsell': 0,
    'volbal': 0,
    'active': 1,
    'dividend': 0,
    'period': 4,
    'grade': 'A1'
}
sqlIns = text("""
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (:name, :date, :volbuy, :price, :volsell, :volbal, :active, :dividend, :period, :grade)
""")
result = const.execute(sqlIns, rcd)
result.rowcount

1

In [68]:
#name = 'CPF'

In [80]:
show_buy(const, name)

,name,date,shares,price,active,period,grade
0,CPF,2025-08-21,0,24.10,1,4,A1


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [46]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,CPF,0.5500,0.0000,1.0000,0.0000,1.5500,0,2025-08-29,2025-09-12,,1


In [52]:
update_dividend(const, name, qty)

'Records updated = 1'

### In case of totally new dividend

In [75]:
rcd = ['CPF', 0.0, 0, 1.00, 0, 100, 4000, '2023-08-29', '2023-09-12', '', 1]
rcd

['CPF', 0.0, 0, 1.0, 0, 100, 4000, '2023-08-29', '2023-09-12', '', 1]

In [87]:
rcd = {
    'name': 'CPF',
    'q4': '0',
    'q3': '0',
    'q2': '1.00',
    'q1': '0',    
    'dividend': '1.00',      
    'shares': 4000,
    'xdate': '2025-08-29',
    'paiddate': '2025-09-12',
    'kind': ' ',
    'actual': 1
}
sqlIns = text("""
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (:name, :q4, :q3, :q2, :q1, :dividend, :shares, :xdate, :paiddate, :kind, :actual)
""")
print(sqlIns)


INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (:name, :q4, :q3, :q2, :q1, :dividend, :shares, :xdate, :paiddate, :kind, :actual)



In [89]:
result = const.execute(sqlIns, rcd)
result.rowcount

1

In [54]:
show_dividend(const, name)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,CPF,0.5500,0.0000,1.0000,0.0000,1.5500,"5,000",2025-08-29,2025-09-12,,1


### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [60]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,152,CPF,24.40,23.90,B,24.10,24.30,"4,000.00",0.00,0.00,"4,000",-4,4,0,0.000000,0.000000,,SET50


In [62]:
#
new_buy_target = 24.1
new_sell_target = 24.30
new_buy_qty = 5_000
print(transaction, name, qty, buy_price, new_buy_target, new_sell_target, new_buy_qty) 

B CPF 5000 24.1 24.1 24.3 5000


In [68]:
# Example usage
update_stock(conlite, name, qty, buy_price, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [70]:
show_stocks(conlite, name)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,152,CPF,24.40,23.90,B,24.10,24.30,"4,000.00",0.00,24.10,"5,000",-4,4,"5,000",0.000000,0.000000,,SET50


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

### =================================================

In [173]:
#price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '{}' \
ORDER BY period, buy.name".format(yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-08-21' ORDER BY period, buy.name


In [175]:
df_buy = pd.read_sql(sql, const)
df_buy

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,14.80,"44,400.00","17,760.00","-26,640.00",-60.00
1,1,A1,AWC,2023-06-15,"9,000",4.96,2.40,"44,640.00","21,600.00","-23,040.00",-51.61
2,1,A2,JMART,2023-01-20,"6,800",33.00,8.45,"224,400.00","57,460.00","-166,940.00",-74.39
3,1,C2,ORI,2022-04-07,"50,000",9.00,2.26,"450,000.00","113,000.00","-337,000.00",-74.89
4,1,C2,PTTGC,2021-03-17,"6,000",64.75,28.00,"388,500.00","168,000.00","-220,500.00",-56.76
5,1,C2,SCC,2021-09-20,600,405.00,227.00,"243,000.00","136,200.00","-106,800.00",-43.95
6,1,A3,SINGER,2023-01-19,"6,000",24.80,5.30,"148,800.00","31,800.00","-117,000.00",-78.63
7,2,A1,AIMIRT,2023-08-17,"17,500",10.60,9.60,"185,500.00","168,000.00","-17,500.00",-9.43
8,2,C1,CPNREIT,2022-08-16,"55,000",18.00,11.40,"990,000.00","627,000.00","-363,000.00",-36.67
9,2,C1,DIF,2020-08-01,"50,000",12.17,8.20,"608,500.00","410,000.00","-198,500.00",-32.62


In [177]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [ ]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

In [120]:
df_buy.to_csv(output_file, header=True, index=False)
df_buy.to_csv(icd_file, header=True, index=False)
df_buy.to_csv(god_file, header=True, index=False)
df_buy.to_csv(osd_file, header=True, index=False)

In [122]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

df_buy.to_excel(xsl_file, index=False)